In [1]:
import ipywidgets as widgets
from IPython.display import display
from tkinter import Tk, filedialog
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from pydub.playback import play
import numpy as np
from pathlib import Path
import tempfile
import os
import subprocess

def change_audio_speed_and_pitch_ffmpeg(input_path, output_path, speed=1.0, pitch_semitones=0):
    # ffmpeg 명령어를 사용하여 오디오 속도 및 피치 변경
    command = [
        'ffmpeg',
        '-i', input_path,
        output_path
    ]
    subprocess.run(command, check=True)

def select_file_and_extract_audio():
    # tkinter 설정
    root = Tk()
    root.withdraw()  # Tkinter 창 숨기기
    
    # 파일 선택 다이얼로그 열기
    file_path = filedialog.askopenfilename(filetypes=[("MKV files", "*.mkv")])
    
    if not file_path:
        print("No file selected")
        return
    
    # 입력받은 경로를 Path 객체로 변환
    input_path = Path(file_path)
    
    # MKV 파일이 존재하는지 확인
    if not input_path.is_file():
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    # 출력 경로 설정 (입력 파일과 동일한 디렉토리에 .wav 확장자로 저장)
    output_path = input_path.with_suffix('.wav')
    
    # MKV 파일에서 오디오 추출 (임시 파일에 저장)
    video = VideoFileClip(str(input_path))
    temp_audio_path = tempfile.mktemp(suffix='.wav')
    video.audio.write_audiofile(temp_audio_path)
    
    # ffmpeg를 사용하여 오디오 속도 변경 및 피치 조정
    change_audio_speed_and_pitch_ffmpeg(temp_audio_path, output_path, speed=0.5, pitch_semitones=0)  # 예시로 한 옥타브 올림
    
    # 임시 파일 삭제
    os.remove(temp_audio_path)
    
    print(f"Audio has been successfully extracted and saved to {output_path}")

# 버튼 생성
button = widgets.Button(description="Select MKV File and Extract Audio")
button.on_click(lambda x: select_file_and_extract_audio())

# 버튼 표시
display(button)

Button(description='Select MKV File and Extract Audio', style=ButtonStyle())